# Imports

In [1]:
import os
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pathlib import Path

In [66]:
import pandas_ta as ta

# Parameters

In [2]:
total_obs=21*12*8-4

In [3]:
backtest_start=pd.to_datetime('2010-01-01')
backtest_end=pd.to_datetime('2016-01-01')

In [4]:
fwd_test_start=pd.to_datetime('2016-01-01')
fwd_test_end=pd.to_datetime('2018-01-01')

# List of SP500 Companies

In [5]:
url="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [6]:
df=pd.read_html(url,header=0)[0]

In [7]:
sp500_comp=list(df['Symbol'])

In [8]:
sp500_comp[:10]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

# Data of Companies in SP500

In [9]:
equity_csv_path=Path("../Data_Store/wiki_prices.csv")

In [10]:
equity_data=pd.read_csv(
    equity_csv_path,
    parse_dates=['date'],
    index_col=['date','ticker']
).sort_index()

In [11]:
equity_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15389314 entries, (Timestamp('1962-01-02 00:00:00'), 'ARNC') to (Timestamp('2018-03-27 00:00:00'), 'ZUMZ')
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   open         float64
 1   high         float64
 2   low          float64
 3   close        float64
 4   volume       float64
 5   ex-dividend  float64
 6   split_ratio  float64
 7   adj_open     float64
 8   adj_high     float64
 9   adj_low      float64
 10  adj_close    float64
 11  adj_volume   float64
dtypes: float64(12)
memory usage: 1.4+ GB


In [12]:
cols_to_include=['adj_open','adj_high','adj_low','adj_close','adj_volume']

In [13]:
equity_data=equity_data[cols_to_include]

In [14]:
available_ticker=equity_data.index.get_level_values("ticker")

In [15]:
available_ticker

Index(['ARNC', 'BA', 'CAT', 'DD', 'DIS', 'GE', 'HPQ', 'IBM', 'KO', 'ARNC',
       ...
       'ZEN', 'ZEUS', 'ZGNX', 'ZION', 'ZIOP', 'ZIXI', 'ZNGA', 'ZOES', 'ZTS',
       'ZUMZ'],
      dtype='object', name='ticker', length=15389314)

In [16]:
equity_data=equity_data[available_ticker.isin(sp500_comp)]

In [18]:
data=equity_data[(equity_data.index.get_level_values("date")>=backtest_start)
                  &(equity_data.index.get_level_values("date")<=fwd_test_end)]

In [19]:
nobs_per_group=data.groupby("ticker").size()
keep_ticker=nobs_per_group[nobs_per_group==total_obs].index

In [20]:
idx=pd.IndexSlice

In [22]:
data=data.loc[idx[:,keep_ticker],:]

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 756512 entries, (Timestamp('2010-01-04 00:00:00'), 'A') to (Timestamp('2017-12-29 00:00:00'), 'ZBRA')
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   adj_open    756512 non-null  float64
 1   adj_high    756512 non-null  float64
 2   adj_low     756512 non-null  float64
 3   adj_close   756512 non-null  float64
 4   adj_volume  756512 non-null  float64
dtypes: float64(5)
memory usage: 32.5+ MB


In [24]:
data.head()

adj_open   adj_high    adj_low  adj_close  adj_volume
date       ticker                                                        
2010-01-04 A       21.415535  21.579273  21.239108  21.354133   2729300.0
           ACGL    23.936667  24.066667  23.916667  23.983333   1604400.0
           ACN     35.020500  35.594053  35.003630  35.484403   3650100.0
           ADI     25.674932  25.997988  25.529556  25.578015   2102700.0
           ADM     25.917474  26.213862  25.793979  25.909241   3472500.0

In [25]:
data.tail()

adj_open  adj_high  adj_low  adj_close  adj_volume
date       ticker                                                    
2017-12-29 XEL        48.11     48.29    48.07      48.11   2765244.0
           XOM        84.00     84.20    83.64      83.64   8463522.0
           YUM        82.64     82.71    81.59      81.61   1343728.0
           ZBH       121.75    121.95   120.62     120.67   1021850.0
           ZBRA      104.33    104.94   103.28     103.80    158534.0

In [26]:
data

adj_open    adj_high     adj_low   adj_close  adj_volume
date       ticker                                                            
2010-01-04 A        21.415535   21.579273   21.239108   21.354133   2729300.0
           ACGL     23.936667   24.066667   23.916667   23.983333   1604400.0
           ACN      35.020500   35.594053   35.003630   35.484403   3650100.0
           ADI      25.674932   25.997988   25.529556   25.578015   2102700.0
           ADM      25.917474   26.213862   25.793979   25.909241   3472500.0
...                       ...         ...         ...         ...         ...
2017-12-29 XEL      48.110000   48.290000   48.070000   48.110000   2765244.0
           XOM      84.000000   84.200000   83.640000   83.640000   8463522.0
           YUM      82.640000   82.710000   81.590000   81.610000   1343728.0
           ZBH     121.750000  121.950000  120.620000  120.670000   1021850.0
           ZBRA    104.330000  104.940000  103.280000  103.800000    158534.0

[756512 rows x 5 columns]

# Fama-French Factor Data

In [27]:
start=data.index.get_level_values("date")[0]

In [28]:
end=data.index.get_level_values("date")[-1]

In [29]:
url='F-F_Research_Data_5_Factors_2x3_daily'

In [30]:
factor_data=web.DataReader(url,'famafrench',start=start,end=end)[0]

/tmp/ipykernel_13083/748594556.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  factor_data=web.DataReader(url,'famafrench',start=start,end=end)[0]


In [31]:
factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2013 entries, 2010-01-04 to 2017-12-29
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Mkt-RF  2013 non-null   float64
 1   SMB     2013 non-null   float64
 2   HML     2013 non-null   float64
 3   RMW     2013 non-null   float64
 4   CMA     2013 non-null   float64
 5   RF      2013 non-null   float64
dtypes: float64(6)
memory usage: 110.1 KB


In [32]:
factor_data.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2010-01-04,1.69,0.79,1.14,-0.17,0.21,0.0
2010-01-05,0.31,-0.42,1.22,-0.18,0.19,0.0
2010-01-06,0.13,-0.14,0.55,-0.05,0.20,0.0
2010-01-07,0.40,0.25,0.96,-0.66,0.22,0.0
2010-01-08,0.33,0.31,0.02,0.23,-0.40,0.0


In [33]:
factor_data.tail()

,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
2017-12-22,-0.07,-0.19,-0.21,-0.10,0.22,0.004
2017-12-26,-0.07,0.31,-0.07,-0.13,0.61,0.004
2017-12-27,0.05,-0.16,-0.20,0.07,-0.12,0.004
2017-12-28,0.22,0.10,0.05,-0.13,-0.19,0.004
2017-12-29,-0.57,-0.31,0.01,0.16,0.13,0.004


# SP500 Data

In [34]:
data_path=Path("../Data_Store/sp500_data.csv")

In [35]:
data_sp500=pd.read_csv(
    data_path,
    parse_dates=['Date'],
    index_col=['Date']
    
).sort_index()

In [36]:
data_sp500=data_sp500["Close"].rename({"Close":"sp500_close"}).to_frame()

In [37]:
data_sp500=data_sp500[(data_sp500.index>=backtest_start)&(data_sp500.index<=fwd_test_end)]

In [38]:
data_sp500.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2013 entries, 2010-01-04 to 2017-12-29
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   2013 non-null   float64
dtypes: float64(1)
memory usage: 31.5 KB


In [39]:
data_sp500.head()

,Close
Date,
2010-01-04,1132.99
2010-01-05,1136.52
2010-01-06,1137.14
2010-01-07,1141.69
2010-01-08,1144.98


In [40]:
data_sp500.tail()

,Close
Date,
2017-12-22,2683.34
2017-12-26,2680.50
2017-12-27,2682.62
2017-12-28,2687.54
2017-12-29,2673.61


# Feature Engineering

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 756512 entries, (Timestamp('2010-01-04 00:00:00'), 'A') to (Timestamp('2017-12-29 00:00:00'), 'ZBRA')
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   adj_open    756512 non-null  float64
 1   adj_high    756512 non-null  float64
 2   adj_low     756512 non-null  float64
 3   adj_close   756512 non-null  float64
 4   adj_volume  756512 non-null  float64
dtypes: float64(5)
memory usage: 32.5+ MB


In [42]:
data=data.rename(columns=lambda x: x.replace("adj_",""))

## Dollar Volume

In [43]:
data["dollar_volume"]=data[["close","volume"]].prod(axis=1)

In [44]:
for t in [7,15,21]:
    data[f"dollar_volume_{t}d"]=(data.dollar_volume.
                                 groupby("ticker").
                                 rolling(window=t).
                                 mean()).values

In [55]:
data["dollar_volume_rank"]=(data
                            .groupby("date")
                            .dollar_volume_21d
                            .rank(ascending=False))

## Returns

In [50]:
q=0.0001

In [51]:
for t in [1,7,15,21]:
    data[f"returns_{t}d"]=(data
                           .groupby("ticker")
                           .close
                           .pct_change()
                           .pipe(lambda x: x.clip(lower=x.quantile(q),upper=x.quantile(1-q)))
                           .add(1)
                           .pow(1/t)
                           .sub(1))

## Lagged Returns

In [54]:
for t in [1,7,15,21]:
    for lag in [1,3,5,7]:
        data[f"returns_{t}d_lag{lag}"]=(data
                                        .groupby("ticker")[f"returns_{t}d"]
                                        .shift(lag))

## Momentum

In [57]:
for t in [7,15,21]:
    data[f"momentum_{t}d"]=(data.
                            groupby("ticker")
                            .returns_1d
                            .rolling(window=t)
                            .mean()
                            .shift(1)).values

## Lagged Momentum

In [59]:
for t in [7,15,21]:
    for lag in [1,3,5,7]:
        data[f"momentum_{t}d_lag{lag}"]=(data.
                                         groupby("ticker")[f"momentum_{t}d"].
                                         shift(lag))

## Volatility

In [60]:
for t in [7,15,21]:
    data[f"volatility_{t}d"]=(data
                              .groupby("ticker")
                              .returns_1d
                              .rolling(window=t)
                              .std()
                              .shift(1)).values

## Lagged Volatility

In [61]:
for t in [7,15,21]:
    for lag in [1,3,5,7]:
        data[f"volatility_{t}d_lag{lag}"]=(data
                                           .groupby("ticker")[f"volatility_{t}d"]
                                           .shift(lag))

## SMA-Simple Moving Average

In [62]:
for t in [7,15,21]:
    data[f"sma_{t}d"]=(data
                       .groupby("ticker")
                       .close
                       .rolling(window=t)
                       .mean()
                       .shift(1)).values

## Min Price

In [64]:
for t in [7,15,21]:
    data[f"min_price_{t}d"]=(data
                             .groupby("ticker")
                             .close
                             .rolling(window=t)
                             .min()).values

## Max Price

In [65]:
for t in [7,15,21]:
    data[f"max_price_{t}d"]=(data
                             .groupby("ticker")
                             .close
                             .rolling(window=t)
                             .max()).values

## Relative Strength Index (RSI)

In [67]:
data["rsi"]=(data
             .groupby("ticker")
             .close
             .transform(lambda x: ta.rsi(x,length=14)))

## Bollinger Bands

In [85]:
def compute_bollinger_bands(group):
    bb=ta.bbands(group['close'],length=20)
    bb=bb.rename(columns={"BBL_20_2.0":"bb_lower","BBU_20_2.0":"bb_upper"})
    bb=bb[["bb_lower","bb_upper"]]
    return group.join(bb)

In [86]:
data=data.groupby("ticker",group_keys=False).apply(compute_bollinger_bands)

In [88]:
data["bb_upper"]=data.bb_upper.sub(data.close).div(data.bb_upper).apply(np.log1p)

In [89]:
data["bb_lower"]=data.close.sub(data.bb_lower).div(data.close).apply(np.log1p)

## Average True Range (ATR)

In [93]:
def compute_avg_true_range(group):
    atr=ta.atr(high=group.high,low=group.low,close=group.close,length=14).to_frame("avg_true_range")
    return group.join(atr)

In [94]:
data=data.groupby("ticker",group_keys=False).apply(compute_avg_true_range)

## Moving Average Convergence Divergence (MACD)

In [97]:
def compute_macd(group):
    macd=ta.macd(group.close,fast=12,slow=26,signal=9)
    macd=macd.rename(columns={"MACD_12_26_9":"macd","MACDs_12_26_9":"macd_signal","MACDh_12_26_9":"macd_hist"})
    return group.join(macd)

In [98]:
data=data.groupby("ticker",group_keys=False).apply(compute_macd)